<h1>Interactive Visual Analytics with Dash Application and Plotly<h1>

In [2]:
pip install dash

  Using cached dash-2.16.1-py3-none-any.whl (10.2 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

In [98]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [255]:
df.rename(columns={"Flight Number": "Flight_Number", "Launch Site": "Launch_Site", "class": "Class", "Payload Mass (kg)": "Payload_Mass_(kg)", "Booster Version": "Booster_Version", "Booster Version Category": "Booster_Version_category"}, inplace=True)
df.head(5)
#print(len(df['Flight_Number']))

,Unnamed: 0,Flight_Number,Launch_Site,Class,Payload_Mass_(kg),Booster_Version,Booster_Version_category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [256]:
# Initialize the app
app = Dash(__name__)


# App layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
                                html.P("Select launch site"),
                                html.Div(dcc.Dropdown(id='site_dropdown',
                                                      options=['ALL', 'CCAFS SLC-40', 'CCAFS LC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
                                                      value='ALL',
                                                      searchable=True,
                                                      clearable=False,
                                                     ) 
                                        ),
                                
                                html.Br(),

                                html.Div(dcc.Graph(id='launch_success_pie', figure=fig)),
       
                                html.Br(),
                                
                                html.P("Select payload mass"),
                                html.Div(dcc.RangeSlider(id='payload_rangeslider',
                                                         min = 0, max = 10000, step=100,
                                                         marks={0: '0 KG',
                                                                1000:'1000 KG',
                                                                2000: '2000 KG', 
                                                                3000:'3000 KG', 
                                                                4000:'4000 KG', 
                                                                5000:'5000 KG', 
                                                                6000:'6000 KG', 
                                                                7000:'7000 KG', 
                                                                8000:'8000 KG', 
                                                                9000:'9000 KG', 
                                                                10000: '10000 KG',},
                                                         value=[int(df['Payload_Mass_(kg)'].min()), int(df['Payload_Mass_(kg)'].max())]
                                                        )
                                        ),
                                html.Br(),

                                html.Div(dcc.Graph(id='payload_site_scatter', figure=fig)),
       
                                html.Br(),  
                                
                               ]
                    )

@app.callback(
    Output('launch_success_pie', 'figure'),
    Input('site_dropdown', 'value')
)
def update_pie(site_dropdown):
    df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
    df.rename(columns={"Flight Number": "Flight_Number", "Launch Site": "Launch_Site", "class": "Class", "Payload Mass (kg)": "Payload_Mass_(kg)", "Booster Version": "Booster_Version", "Booster Version Category": "Booster_Version_category"}, inplace=True)
    
    if (site_dropdown !='ALL'):
        df_site= df.loc[df['Launch_Site'] ==site_dropdown].reset_index(drop=True)
        df_site.loc[:,'Flight_Number']=1
        
        fig= px.pie(df_site, values='Flight_Number',names= 'Class', title=str(site_dropdown))
        return fig
    else:
        fig = px.pie(df, values='Class', names='Launch_Site', title='All launch sites landing outcomes')
        return fig


@app.callback(
    Output("payload_site_scatter", "figure"), 
    [Input("payload_rangeslider", "value"),
     Input('site_dropdown', 'value')]
)
def update_scatter(payload_rangeslider, site_dropdown):
    df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
    df.rename(columns={"Flight Number": "Flight_Number", "Launch Site": "Launch_Site", "class": "Class", "Payload Mass (kg)": "Payload_Mass_(kg)", "Booster Version": "Booster_Version", "Booster Version Category": "Booster_Version_category"}, inplace=True)
    
    low, high = payload_rangeslider
    
    if (site_dropdown !='ALL'):
        df_site= df.loc[df['Launch_Site'] ==site_dropdown].reset_index(drop=True)
        mask = (df_site['Payload_Mass_(kg)'] > low) & (df_site['Payload_Mass_(kg)'] < high)
        fig = px.scatter(df_site[mask], x="Payload_Mass_(kg)", y="Class", color="Booster_Version_category", title=str(site_dropdown))
        return fig
    else:
        mask = (df['Payload_Mass_(kg)'] > low) & (df['Payload_Mass_(kg)'] < high)
        fig = px.scatter(df[mask], x="Payload_Mass_(kg)", y="Class", color="Booster_Version_category", title="Correlation betn Payload Mass and Landing Outcome")
        return fig
    
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [257]:

df_site= df.loc[df['Launch_Site'] =='KSC LC-39A'].reset_index(drop=True)
#print(len(index_site))
#index_site= [i for i in range(1, len(df_site)+1)]
#df_site.loc[:,'Flight_Number']=index_site
    

<h3>Thank You<h3>